In [35]:
import os, sys, glob
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Conv1D
import tensorflow.keras.utils as conv_utils 
import tensorflow as tf
from tensorflow.keras.regularizers import l2

In [43]:
def asymmetric_temporal_padding(x, left_pad=1, right_pad=1):
    '''Pad the middle dimension of a 3D tensor
    with "left_pad" zeros left and "right_pad" right.
    '''
    pattern = [[0, 0], [left_pad, right_pad], [0, 0]]
    return tf.pad(x, pattern)


class CausalAtrousConvolution1D(Conv1D):
    def __init__(self, filters, kernel_size, 
                 init='glorot_uniform', activation=None,
                 padding='valid', strides=1, 
                 dilation_rate=1, bias_regularizer=None,
                 activity_regularizer=None, kernel_constraint=None,
                 bias_constraint=None, use_bias=True, causal=False, **kwargs):
        
        super(CausalAtrousConvolution1D, self).__init__(filters,
                                                        kernel_size=kernel_size,
                                                        strides=strides,
                                                        padding=padding,
                                                        dilation_rate=dilation_rate,
                                                        activation=activation,
                                                        use_bias=use_bias,
                                                        kernel_initializer=init,
                                                        activity_regularizer=activity_regularizer,
                                                        bias_regularizer=bias_regularizer,
                                                        kernel_constraint=kernel_constraint,
                                                        bias_constraint=bias_constraint,
                                                        **kwargs)

        self.causal = causal
        if self.causal and padding != 'valid':
            raise ValueError("Causal mode dictates border_mode=valid.")

    def compute_output_shape(self, input_shape):
        input_length = input_shape[1]

        if self.causal:
            input_length += self.dilation_rate[0] * (self.kernel_size[0] - 1)

        length = conv_output_length(input_length,
                                    self.kernel_size[0],
                                    self.padding,
                                    self.strides[0],
                                    dilation=self.dilation_rate[0])

        return (input_shape[0], length, self.filters)

    def call(self, x):
        if self.causal:
            x = asymmetric_temporal_padding(x, self.dilation_rate[0] * (self.kernel_size[0] - 1), 0)
        return super(CausalAtrousConvolution1D, self).call(x)

In [46]:
def residual_block(x, nb_filters, i, res_l2=0, use_bias=False):
    x_org=x
    f = CausalAtrousConvolution1D(nb_filters, 2, dilation_rate=2 ** i, 
                                  padding='valid', causal=True, use_bias=use_bias,
                                  name='dilated_conv_%d_tanh' % (2 ** i), 
                                  activation='tanh',
                                  kernel_regularizer=l2(res_l2))(x)
    
    g = CausalAtrousConvolution1D(nb_filters, 2, dilation_rate=2 ** i, 
                                  padding='valid', causal=True, use_bias=use_bias,
                                  name='dilated_conv_%d_sigmoid' % (2 ** i), 
                                  activation='sigmoid',
                                  kernel_regularizer=l2(res_l2))(x)

    z = layers.Multiply()([f,g])
    z = layers.Convolution1D(nb_filters, 1, 
                             padding='same', 
                             use_bias=use_bias,
                             kernel_regularizer=l2(res_l2))(x)

    y = layers.Add()([x_org, z])
    
    return z, y

In [48]:
x = layers.Input(shape=(1200,1))
z1, y1 = residual_block(x, 10, 1)
z2, y2 = residual_block(y1, 10, 2)
z3, y3 = residual_block(y2,10, 4)
model = models.Model(x, y3)

In [49]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 1200, 1)]    0                                            
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 1200, 10)     10          input_14[0][0]                   
__________________________________________________________________________________________________
add_4 (Add)                     (None, 1200, 10)     0           input_14[0][0]                   
                                                                 conv1d_4[0][0]                   
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 1200, 10)     100         add_4[0][0]                  